In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso

from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn import linear_model
import sklearn.metrics as metrics
from sklearn.tree import DecisionTreeRegressor 
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Ridge

#cross validation
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV

#use of pipelines
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

#supress warnings
import warnings
warnings.filterwarnings("ignore")

from sklearn import set_config
set_config(display="diagram")

#timer
import time

#Set seed
seed = 32

In [4]:
#Load data and get data subsets
os.chdir("C:/Users/17176/Desktop/DAAN 888 - Capstone/Project")
final_data = pd.read_csv("final_data.csv")
final_data = final_data.drop('Unnamed: 0', axis = 1)

#we are interested only in the Animals that are adopted
#So, filter the subset based on 'Outcome Type' == "Adoption"
filter_adoption = final_data['Outcome Type'] == "Adoption"
adoption_data = final_data[filter_adoption] 

#drop the Outcome Type
adoption_data = adoption_data.drop(['Outcome Type','Intake Date'], axis =1)
adoption_data

#prepare the list of the features to be used for modeeling i.e independent variables
#and the target feature i.e Duration
feat_cols = adoption_data.columns.to_list()
feat_cols.remove('Duration')

#Repeat for cat adoptions
cat_adoption_data = adoption_data[adoption_data["Animal Type_Cat"] == 1]
cat_adoption_data = cat_adoption_data.drop("Animal Type_Cat", axis = 1)
cat_adoption_data = cat_adoption_data.drop("Animal Type_Dog", axis = 1)

feat_cols_cats = cat_adoption_data.columns.to_list()
feat_cols_cats.remove('Duration')

#Repeat for dog adoptions
dog_adoption_data = adoption_data[adoption_data["Animal Type_Dog"] == 1]
dog_adoption_data = dog_adoption_data.drop("Animal Type_Dog", axis = 1)
dog_adoption_data = dog_adoption_data.drop("Animal Type_Cat", axis = 1)

feat_cols_dogs = dog_adoption_data.columns.to_list()
feat_cols_dogs.remove('Duration')

# Apply Results from Feature Selection to Each Adoption Subset

In [5]:
#Get dataset for selected features -- all adoptions
#remove irrelevant variables
adoption_data_select = adoption_data.drop(['Sex_Female','Intake Status_Unknown','Intake Type_Public Assist',
                                          'Sex_Unknown','Intake Type_Euthanasia Request','Intake Type_Abandoned',
                                          'Outcome Status_Unknown'], axis =1)
#Move duration to the first column
first_col_all = adoption_data_select.pop('Duration')
adoption_data_select.insert(0, 'Duration', first_col_all)

adoption_data_select

,Duration,Outcome Age,Intake Type_Owner Surrender,Intake Type_Stray,Intake Condition_0,Intake Condition_1,Animal Type_Cat,Animal Type_Dog,Intake Status_Intact,Intake Status_Sterile,Sex_Male,Outcome Status_Intact,Outcome Status_Sterile,New Breed_Mixed,New Breed_Multi,New Breed_Single
12,3.103873,14.916529,1,0,1,0,0,1,0,1,1,0,1,1,0,0
14,1.999925,15.114757,0,0,0,1,0,1,0,1,1,0,1,0,0,1
23,2.621074,14.783650,0,1,1,0,0,1,0,1,1,0,1,0,1,0
33,3.444960,14.871002,0,1,1,0,0,1,0,1,0,0,1,1,0,0
49,1.272469,14.008931,0,1,1,0,0,1,0,1,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126752,1.026284,6.838251,1,0,1,0,1,0,1,0,1,0,1,1,0,0
126753,1.026284,6.838251,1,0,1,0,1,0,1,0,0,0,1,1,0,0
126754,1.026284,6.838251,1,0,1,0,1,0,1,0,0,0,1,1,0,0
126756,1.026284,14.719901,1,0,0,1,0,1,0,1,1,0,1,1,0,0


In [6]:
#Get dataset for selected features -- cat adoptions
#use subset of relevant features determined during feature selection
cat_adoption_data_select = cat_adoption_data[["Duration","Outcome Age","Intake Type_Owner Surrender",
                                      "Intake Condition_0","Intake Condition_1","Intake Status_Intact",
                                      "Intake Status_Sterile","Outcome Status_Sterile","Intake Type_Stray"]]

cat_adoption_data_select

,Duration,Outcome Age,Intake Type_Owner Surrender,Intake Condition_0,Intake Condition_1,Intake Status_Intact,Intake Status_Sterile,Outcome Status_Sterile,Intake Type_Stray
56,2.920820,14.173198,0,1,0,0,1,1,1
57,1.604705,14.486417,1,1,0,0,1,1,0
73,3.801525,14.266655,0,1,0,0,1,1,1
81,1.920468,14.364476,0,1,0,0,1,1,1
90,2.562346,14.138201,1,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...
126738,1.457400,5.359999,1,1,0,1,0,1,0
126743,1.604705,9.888372,0,0,1,0,1,1,1
126752,1.026284,6.838251,1,1,0,1,0,1,0
126753,1.026284,6.838251,1,1,0,1,0,1,0


In [7]:
#Get dataset for selected features -- dog adoptions
#use subset of relevant features determined during feature selection
dog_adoption_data_select = dog_adoption_data[["Duration","Outcome Age","Intake Type_Owner Surrender",
                                              "Intake Condition_0","Intake Condition_1","Intake Status_Intact",
                                              "Intake Status_Sterile","Outcome Status_Intact","Intake Type_Public Assist",
                                              "Intake Type_Stray","Sex_Female"]]

dog_adoption_data_select

,Duration,Outcome Age,Intake Type_Owner Surrender,Intake Condition_0,Intake Condition_1,Intake Status_Intact,Intake Status_Sterile,Outcome Status_Intact,Intake Type_Public Assist,Intake Type_Stray,Sex_Female
12,3.103873,14.916529,1,1,0,0,1,0,0,0,0
14,1.999925,15.114757,0,0,1,0,1,0,1,0,0
23,2.621074,14.783650,0,1,0,0,1,0,0,1,0
33,3.444960,14.871002,0,1,0,0,1,0,0,1,1
49,1.272469,14.008931,0,1,0,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...
126740,1.457400,8.128098,1,1,0,1,0,0,0,0,0
126750,0.663872,5.636692,1,1,0,1,0,0,0,0,0
126751,0.663872,5.636692,1,1,0,1,0,0,0,0,0
126756,1.026284,14.719901,1,0,1,0,1,0,0,0,0


# Define Common functions

In [8]:
def TestTrainSplit(data):
    #Get X and y values for each subset and set of features
    y = data['Duration']
    features = data.columns.to_list()
    features.remove('Duration')
    X = data[features]
    
    # split into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)
    return X_train, X_test, y_train, y_test

def PerformModelling(X_train, X_test, y_train, y_test, model, model_name, data_info):
    start = time.time()
    print("********************************")
    print(model)
    print(model_name, " - ", data_info)
    print("Shape of train data",X_train.shape)
    print("Shape of test data",X_test.shape)
    print("********************************")
    
    
    #Scale the data
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    #Train a lasso regression model
    model.fit(X_train, y_train)
    
    #Use the lasso model to predict duration
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    
    compute_metrics(model_name, data_info, y_train, train_pred, y_test, test_pred)
    end = time.time()
    print("Time taken for modelling: ", end - start)
    print()
    
# evaluate predictions
def compute_metrics(model,data,y_train_set, y_train_pred, y_test_set, y_test_pred):
    print("Model:",model)
    print("Data:",data)
    mae_train = mean_absolute_error(y_train_set, y_train_pred)
    print('MAE Train: %.3f' % mae_train)
    mae_test = mean_absolute_error(y_test_set, y_test_pred)
    print('MAE Test: %.3f' % mae_test)
    mse_train = mean_squared_error(y_train_set, y_train_pred)
    print('MSE Train: %.3f' % mse_train)
    mse_test = mean_squared_error(y_test_set, y_test_pred)
    print('MSE Test: %.3f' % mse_test)
    r2_train = r2_score(y_train_set, y_train_pred)
    print('R-squared Train: %.3f' % r2_train)
    r2_test = r2_score(y_test_set, y_test_pred)
    print('R-squared Test: %.3f' % r2_test) 
    
def GridCVforTuning(model, space, X, y):
    start = time.time()
    print("Identifying the best Hyperparameters.....")
    # define evaluation
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    # define search
    search = GridSearchCV(model, space, scoring='r2', n_jobs=-1, cv=cv)
    # execute search
    result = search.fit(X, y)
    # summarize result
    print('Best Score: %s' % result.best_score_)
    print('Best Hyperparameters: %s' % result.best_params_)
    end = time.time()
    print("Time taken for modelling: ", end - start)
    print()
    return result.best_params_

    

# Prepare the data for modelling
Divide the different subsets - all adoptions, cat adoptions and dog adoption data 
into test and train sets.

In [9]:
################################################################
###################Consider all features #######################
#All adoptions
print("Shape of {} ".format("adoption_data"),adoption_data.shape)
# split into train and test sets
X_train, X_test, y_train, y_test = TestTrainSplit(adoption_data)


#All CAT adoptions
print("Shape of {} ".format("cat_adoption_data"),cat_adoption_data.shape)
# split into train and test sets
X_cat_train, X_cat_test, y_cat_train, y_cat_test = TestTrainSplit(cat_adoption_data)

    
#All DOG adoptions
print("Shape of {} ".format("dog_adoption_data"),dog_adoption_data.shape)
# split into train and test sets
X_dog_train, X_dog_test, y_dog_train, y_dog_test = TestTrainSplit(dog_adoption_data)
    
################################################################
###################selected features only#######################
#All adoptions
print("Shape of {} ".format("adoption_data_select"),adoption_data_select.shape) 
# split into train and test sets
X_fs_train, X_fs_test, y_fs_train, y_fs_test = TestTrainSplit(adoption_data_select)


#All CAT adoptions
print("Shape of {} ".format("cat_adoption_data_select"),cat_adoption_data_select.shape)
# split into train and test sets
X_cat_fs_train, X_cat_fs_test, y_cat_fs_train, y_cat_fs_test =TestTrainSplit(cat_adoption_data_select)

    
#All DOG adoptions
print("Shape of {}".format("dog_adoption_data_select"),dog_adoption_data_select.shape)
# split into train and test sets
X_dog_fs_train, X_dog_fs_test, y_dog_fs_train, y_dog_fs_test = TestTrainSplit(dog_adoption_data_select)   

Shape of adoption_data  (60407, 23)
Shape of cat_adoption_data  (24154, 21)
Shape of dog_adoption_data  (36253, 21)
Shape of adoption_data_select  (60407, 16)
Shape of cat_adoption_data_select  (24154, 9)
Shape of dog_adoption_data_select (36253, 11)


# Modelling

# Ridge Regression
1. Tried to fit the Ridge regression on the given data
2. Identify the best metrics
3. Now that we know which metrics are best, defined a model with optimised params
4. Perform modelling 

In [24]:
# define model
model = Ridge()
# define search space
space = dict()
space['solver'] = ['svd', 'cholesky', 'lsqr', 'sag']
space['alpha'] = [0, 0.5, 1, 5, 10]
#space['fit_intercept'] = [True, False]

print("*************All Adoptions***************")
print(X_fs_train.shape)
print(y_fs_train.shape)
ridge_all_best = GridCVforTuning(model, space, X_fs_train, y_fs_train)


print("*************Cat Adoptions***************")
print(X_cat_fs_train.shape)
print(y_cat_fs_train.shape)
ridge_cat_best = GridCVforTuning(model, space, X_cat_fs_train, y_cat_fs_train)


print("*************Dog Adoptions***************")
print(X_dog_fs_train.shape)
print(y_dog_fs_train.shape)
ridge_dog_best = GridCVforTuning(model, space, X_dog_fs_train, y_dog_fs_train)


*************All Adoptions***************
(42284, 15)
(42284,)
Identifying the best Hyperparameters.....
Best Score: 0.1835646499152666
Best Hyperparameters: {'alpha': 10, 'solver': 'svd'}
Time taken for modelling:  21.0484459400177

*************Cat Adoptions***************
(16907, 8)
(16907,)
Identifying the best Hyperparameters.....
Best Score: 0.14076312532854895
Best Hyperparameters: {'alpha': 1, 'solver': 'sag'}
Time taken for modelling:  15.69871187210083

*************Dog Adoptions***************
(25377, 10)
(25377,)
Identifying the best Hyperparameters.....
Best Score: 0.14043957447310004
Best Hyperparameters: {'alpha': 1, 'solver': 'sag'}
Time taken for modelling:  11.374798774719238



In [25]:
ridge_all = Ridge(alpha=10, fit_intercept = True , solver='svd')
ridge_cat = Ridge(alpha=1, fit_intercept = True , solver='sag')
ridge_dog = Ridge(alpha=1,  fit_intercept = True , solver='sag')

In [26]:
#####################################################
##############Modelling on all features##############
#####################################################
PerformModelling(X_train, X_test, y_train, y_test, ridge_all,
                 "Optimised Ridge", "All adoptions - all features")
PerformModelling(X_cat_train, X_cat_test, y_cat_train, y_cat_test, ridge_cat,
                 "Optimised Ridge", "Cat adoptions - all features")
PerformModelling(X_dog_train, X_dog_test, y_dog_train, y_dog_test, ridge_dog,
                 "Optimised Ridge", "Dog adoptions - all features")

********************************
Ridge(alpha=10, solver='svd')
Optimised Ridge  -  All adoptions - all features
Shape of train data (42284, 22)
Shape of test data (18123, 22)
********************************
Model: Optimised Ridge
Data: All adoptions - all features
MAE Train: 0.619
MAE Test: 0.624
MSE Train: 0.556
MSE Test: 0.560
R-squared Train: 0.185
R-squared Test: 0.173
Time taken for modelling:  0.08133673667907715

********************************
Ridge(alpha=1, solver='sag')
Optimised Ridge  -  Cat adoptions - all features
Shape of train data (16907, 20)
Shape of test data (7247, 20)
********************************
Model: Optimised Ridge
Data: Cat adoptions - all features
MAE Train: 0.611
MAE Test: 0.608
MSE Train: 0.514
MSE Test: 0.512
R-squared Train: 0.145
R-squared Test: 0.149
Time taken for modelling:  0.19620776176452637

********************************
Ridge(alpha=1, solver='sag')
Optimised Ridge  -  Dog adoptions - all features
Shape of train data (25377, 20)
Shape of 

In [27]:
#####################################################
#########Modelling on selected features##############
#####################################################
PerformModelling(X_fs_train, X_fs_test, y_fs_train, y_fs_test , ridge_all,
                 "Optimised Ridge", "All adoptions - selected features")
PerformModelling(X_cat_fs_train, X_cat_fs_test, y_cat_fs_train, y_cat_fs_test, ridge_cat,
                 "Optimised Ridge", "Cat adoptions - selected features")
PerformModelling(X_dog_fs_train, X_dog_fs_test, y_dog_fs_train, y_dog_fs_test,
                 ridge_dog,
                 "Optimised Ridge", "Dog adoptions - selected features")

********************************
Ridge(alpha=10, solver='svd')
Optimised Ridge  -  All adoptions - selected features
Shape of train data (42284, 15)
Shape of test data (18123, 15)
********************************
Model: Optimised Ridge
Data: All adoptions - selected features
MAE Train: 0.620
MAE Test: 0.624
MSE Train: 0.556
MSE Test: 0.560
R-squared Train: 0.184
R-squared Test: 0.173
Time taken for modelling:  0.06367373466491699

********************************
Ridge(alpha=1, solver='sag')
Optimised Ridge  -  Cat adoptions - selected features
Shape of train data (16907, 8)
Shape of test data (7247, 8)
********************************
Model: Optimised Ridge
Data: Cat adoptions - selected features
MAE Train: 0.613
MAE Test: 0.610
MSE Train: 0.516
MSE Test: 0.514
R-squared Train: 0.142
R-squared Test: 0.145
Time taken for modelling:  0.1406111717224121

********************************
Ridge(alpha=1, solver='sag')
Optimised Ridge  -  Dog adoptions - selected features
Shape of train data

# Lasso Regression

In [20]:
# define model
model =  Lasso()
# define search space
space = dict()
space['alpha'] = [0.1,0.25,0.5,1,10]
#space['fit_intercept'] = [True, False]
#space['normalize'] = [True, False]

print("*************All Adoptions***************")
print(X_fs_train.shape)
print(y_fs_train.shape)
lasso_all_best = GridCVforTuning(model, space, X_fs_train, y_fs_train)

print("*************Cat Adoptions***************")
print(X_cat_fs_train.shape)
print(y_cat_fs_train.shape)
lasso_cat_best = GridCVforTuning(model, space, X_cat_fs_train, y_cat_fs_train)

print("*************Dog Adoptions***************")
print(X_dog_fs_train.shape)
print(y_dog_fs_train.shape)
lasso_dog_best = GridCVforTuning(model, space, X_dog_fs_train, y_dog_fs_train)


*************All Adoptions***************
(42284, 15)
(42284,)
Identifying the best Hyperparameters.....
Best Score: 0.02413400895713519
Best Hyperparameters: {'alpha': 0.1}
Time taken for modelling:  1.5718317031860352

*************Cat Adoptions***************
(16907, 8)
(16907,)
Identifying the best Hyperparameters.....
Best Score: 0.00246147406480803
Best Hyperparameters: {'alpha': 0.1}
Time taken for modelling:  0.6388571262359619

*************Dog Adoptions***************
(25377, 10)
(25377,)
Identifying the best Hyperparameters.....
Best Score: 0.07643569178386252
Best Hyperparameters: {'alpha': 0.1}
Time taken for modelling:  0.8817591667175293



In [21]:
lasso_all = Lasso(alpha=0.1, fit_intercept = True)
lasso_cat = Lasso(alpha=0.1, fit_intercept = True)
lasso_dog = Lasso(alpha=0.1, fit_intercept = True)

In [22]:
#####################################################
#########Modelling on all features##############
#####################################################
PerformModelling(X_train, X_test, y_train, y_test , lasso_all,
                 "Optimised Lasso", "All adoptions - selected features")
PerformModelling(X_cat_train, X_cat_test, y_cat_train, y_cat_test, lasso_cat,
                 "Optimised Lasso", "Cat adoptions - selected features")
PerformModelling(X_dog_train, X_dog_test, y_dog_train, y_dog_test,
                 lasso_dog,
                 "Optimised Lasso", "Dog adoptions - selected features")

********************************
Lasso(alpha=0.1)
Optimised Lasso  -  All adoptions - selected features
Shape of train data (42284, 22)
Shape of test data (18123, 22)
********************************
Model: Optimised Lasso
Data: All adoptions - selected features
MAE Train: 0.666
MAE Test: 0.666
MSE Train: 0.620
MSE Test: 0.618
R-squared Train: 0.091
R-squared Test: 0.088
Time taken for modelling:  0.09461832046508789

********************************
Lasso(alpha=0.1)
Optimised Lasso  -  Cat adoptions - selected features
Shape of train data (16907, 20)
Shape of test data (7247, 20)
********************************
Model: Optimised Lasso
Data: Cat adoptions - selected features
MAE Train: 0.644
MAE Test: 0.640
MSE Train: 0.569
MSE Test: 0.568
R-squared Train: 0.053
R-squared Test: 0.055
Time taken for modelling:  0.03260540962219238

********************************
Lasso(alpha=0.1)
Optimised Lasso  -  Dog adoptions - selected features
Shape of train data (25377, 20)
Shape of test data (1

In [23]:
#####################################################
#########Modelling on selected features##############
#####################################################
PerformModelling(X_fs_train, X_fs_test, y_fs_train, y_fs_test , lasso_all,
                 "Optimised Lasso", "All adoptions - selected features")
PerformModelling(X_cat_fs_train, X_cat_fs_test, y_cat_fs_train, y_cat_fs_test, lasso_cat,
                 "Optimised Lasso", "Cat adoptions - selected features")
PerformModelling(X_dog_fs_train, X_dog_fs_test, y_dog_fs_train, y_dog_fs_test,
                 lasso_dog,
                 "Optimised Lasso", "Dog adoptions - selected features")

********************************
Lasso(alpha=0.1)
Optimised Lasso  -  All adoptions - selected features
Shape of train data (42284, 15)
Shape of test data (18123, 15)
********************************
Model: Optimised Lasso
Data: All adoptions - selected features
MAE Train: 0.666
MAE Test: 0.666
MSE Train: 0.620
MSE Test: 0.618
R-squared Train: 0.091
R-squared Test: 0.088
Time taken for modelling:  0.052158355712890625

********************************
Lasso(alpha=0.1)
Optimised Lasso  -  Cat adoptions - selected features
Shape of train data (16907, 8)
Shape of test data (7247, 8)
********************************
Model: Optimised Lasso
Data: Cat adoptions - selected features
MAE Train: 0.644
MAE Test: 0.640
MSE Train: 0.569
MSE Test: 0.568
R-squared Train: 0.053
R-squared Test: 0.055
Time taken for modelling:  0.015311717987060547

********************************
Lasso(alpha=0.1)
Optimised Lasso  -  Dog adoptions - selected features
Shape of train data (25377, 10)
Shape of test data (1

# Regression Tree

In [16]:
from scipy.stats import randint
model = DecisionTreeRegressor()
space = dict()
space["max_depth"] = [8, 10, 12]
#space["criterion"] = ["gini", "entropy"] #throwing error

print("*************All Adoptions***************")
print(X_fs_train.shape)
print(y_fs_train.shape)
rtree_all_best = GridCVforTuning(model, space, X_fs_train, y_fs_train)

print("*************Cat Adoptions***************")
print(X_cat_fs_train.shape)
print(y_cat_fs_train.shape)
rtree_cat_best = GridCVforTuning(model, space, X_cat_fs_train, y_cat_fs_train)

print("*************Dog Adoptions***************")
print(X_dog_fs_train.shape)
print(y_dog_fs_train.shape)
rtree_dog_best = GridCVforTuning(model, space, X_dog_fs_train, y_dog_fs_train)

*************All Adoptions***************
(42284, 15)
(42284,)
Identifying the best Hyperparameters.....
Best Score: 0.3817120641084851
Best Hyperparameters: {'max_depth': 10}
Time taken for modelling:  1.330820083618164

*************Cat Adoptions***************
(16907, 8)
(16907,)
Identifying the best Hyperparameters.....
Best Score: 0.2631061365375259
Best Hyperparameters: {'max_depth': 12}
Time taken for modelling:  0.3072359561920166

*************Dog Adoptions***************
(25377, 10)
(25377,)
Identifying the best Hyperparameters.....
Best Score: 0.3918219121927141
Best Hyperparameters: {'max_depth': 10}
Time taken for modelling:  0.599707841873169



In [17]:
rtree_all = DecisionTreeRegressor(max_depth = 10)
rtree_cat = DecisionTreeRegressor(max_depth = 12)
rtree_dog = DecisionTreeRegressor(max_depth = 10)

In [18]:
#####################################################
#########Modelling on all features##############
#####################################################
PerformModelling(X_train, X_test, y_train, y_test , rtree_all,
                 "Optimised DecisionTreeRegressor", "All adoptions - selected features")
PerformModelling(X_cat_train, X_cat_test, y_cat_train, y_cat_test, rtree_cat,
                 "Optimised DecisionTreeRegressor", "Cat adoptions - selected features")
PerformModelling(X_dog_train, X_dog_test, y_dog_train, y_dog_test,
                 rtree_dog,
                 "Optimised DecisionTreeRegressor", "Dog adoptions - selected features")

********************************
DecisionTreeRegressor(max_depth=10)
Optimised DecisionTreeRegressor  -  All adoptions - selected features
Shape of train data (42284, 22)
Shape of test data (18123, 22)
********************************
Model: Optimised DecisionTreeRegressor
Data: All adoptions - selected features
MAE Train: 0.443
MAE Test: 0.467
MSE Train: 0.373
MSE Test: 0.414
R-squared Train: 0.453
R-squared Test: 0.389
Time taken for modelling:  0.08333587646484375

********************************
DecisionTreeRegressor(max_depth=12)
Optimised DecisionTreeRegressor  -  Cat adoptions - selected features
Shape of train data (16907, 20)
Shape of test data (7247, 20)
********************************
Model: Optimised DecisionTreeRegressor
Data: Cat adoptions - selected features
MAE Train: 0.461
MAE Test: 0.526
MSE Train: 0.349
MSE Test: 0.455
R-squared Train: 0.419
R-squared Test: 0.244
Time taken for modelling:  0.024159908294677734

********************************
DecisionTreeRegressor

In [19]:
#####################################################
#########Modelling on selected features##############
#####################################################
PerformModelling(X_fs_train, X_fs_test, y_fs_train, y_fs_test , rtree_all,
                 "Optimised DecisionTreeRegressor", "All adoptions - selected features")
PerformModelling(X_cat_fs_train, X_cat_fs_test, y_cat_fs_train, y_cat_fs_test, rtree_cat,
                 "Optimised DecisionTreeRegressor", "Cat adoptions - selected features")
PerformModelling(X_dog_fs_train, X_dog_fs_test, y_dog_fs_train, y_dog_fs_test,
                 rtree_dog,
                 "Optimised DecisionTreeRegressor", "Dog adoptions - selected features")

********************************
DecisionTreeRegressor(max_depth=10)
Optimised DecisionTreeRegressor  -  All adoptions - selected features
Shape of train data (42284, 15)
Shape of test data (18123, 15)
********************************
Model: Optimised DecisionTreeRegressor
Data: All adoptions - selected features
MAE Train: 0.443
MAE Test: 0.466
MSE Train: 0.373
MSE Test: 0.413
R-squared Train: 0.453
R-squared Test: 0.390
Time taken for modelling:  0.05211806297302246

********************************
DecisionTreeRegressor(max_depth=12)
Optimised DecisionTreeRegressor  -  Cat adoptions - selected features
Shape of train data (16907, 8)
Shape of test data (7247, 8)
********************************
Model: Optimised DecisionTreeRegressor
Data: Cat adoptions - selected features
MAE Train: 0.464
MAE Test: 0.519
MSE Train: 0.352
MSE Test: 0.444
R-squared Train: 0.415
R-squared Test: 0.262
Time taken for modelling:  0.017921924591064453

********************************
DecisionTreeRegressor(m

# Adaboost

In [20]:
# define model
model =  AdaBoostRegressor(random_state = seed)
# define search space
space = dict()
space['learning_rate'] = [0.1, 0.5, 1]
space['n_estimators'] = [10, 50, 60]
space['loss'] = ['linear','square','exponential']

print("*************All Adoptions***************")
print(X_fs_train.shape)
print(y_fs_train.shape)
ada_all_best = GridCVforTuning(model, space, X_fs_train, y_fs_train)
                        
print("*************Cat Adoptions***************")
print(X_cat_fs_train.shape)
print(y_cat_fs_train.shape)
ada_cat_best = GridCVforTuning(model, space, X_cat_fs_train, y_cat_fs_train)
                           
print("*************Dog Adoptions***************")
print(X_dog_fs_train.shape)
print(y_dog_fs_train.shape)
ada_dog_best = GridCVforTuning(model, space, X_dog_fs_train, y_dog_fs_train)


*************All Adoptions***************
(42284, 15)
(42284,)
Identifying the best Hyperparameters.....
Best Score: 0.21071047797387948
Best Hyperparameters: {'learning_rate': 0.1, 'loss': 'square', 'n_estimators': 10}
Time taken for modelling:  146.51666402816772

*************Cat Adoptions***************
(16907, 8)
(16907,)
Identifying the best Hyperparameters.....
Best Score: 0.16116996460296268
Best Hyperparameters: {'learning_rate': 0.1, 'loss': 'exponential', 'n_estimators': 50}
Time taken for modelling:  41.81252932548523

*************Dog Adoptions***************
(25377, 10)
(25377,)
Identifying the best Hyperparameters.....
Best Score: 0.22842102028984007
Best Hyperparameters: {'learning_rate': 0.1, 'loss': 'exponential', 'n_estimators': 50}
Time taken for modelling:  68.05509686470032



In [16]:
ada_all = AdaBoostRegressor(learning_rate = 0.1, loss ='square', n_estimators = 10, random_state = seed)
ada_cat = AdaBoostRegressor(learning_rate = 0.1, loss ='exponential', n_estimators = 50, random_state = seed)
ada_dog = AdaBoostRegressor(learning_rate = 0.1, loss ='exponential', n_estimators = 50, random_state = seed)

In [17]:
#####################################################
#########Modelling on all features##############
#####################################################
PerformModelling(X_train, X_test, y_train, y_test , ada_all,
                 "Optimised Adaboost", "All adoptions - selected features")
PerformModelling(X_cat_train, X_cat_test, y_cat_train, y_cat_test, ada_cat,
                 "Optimised Adabost", "Cat adoptions - selected features")
PerformModelling(X_dog_train, X_dog_test, y_dog_train, y_dog_test,
                 ada_dog,
                 "Optimised Adaboost", "Dog adoptions - selected features")

********************************
AdaBoostRegressor(learning_rate=0.1, loss='square', n_estimators=10,
                  random_state=32)
Optimised Adaboost  -  All adoptions - selected features
Shape of train data (42284, 22)
Shape of test data (18123, 22)
********************************
Model: Optimised Adaboost
Data: All adoptions - selected features
MAE Train: 0.589
MAE Test: 0.592
MSE Train: 0.537
MSE Test: 0.538
R-squared Train: 0.213
R-squared Test: 0.205
Time taken for modelling:  0.8897950649261475

********************************
AdaBoostRegressor(learning_rate=0.1, loss='exponential', random_state=32)
Optimised Adabost  -  Cat adoptions - selected features
Shape of train data (16907, 20)
Shape of test data (7247, 20)
********************************
Model: Optimised Adabost
Data: Cat adoptions - selected features
MAE Train: 0.606
MAE Test: 0.603
MSE Train: 0.503
MSE Test: 0.502
R-squared Train: 0.164
R-squared Test: 0.164
Time taken for modelling:  1.3485596179962158

*****

In [18]:
#####################################################
#########Modelling on selected features##############
#####################################################
PerformModelling(X_fs_train, X_fs_test, y_fs_train, y_fs_test , ada_all,
                 "Optimised Adaboost", "All adoptions - selected features")
PerformModelling(X_cat_fs_train, X_cat_fs_test, y_cat_fs_train, y_cat_fs_test, ada_cat,
                 "Optimised Adaboost", "Cat adoptions - selected features")
PerformModelling(X_dog_fs_train, X_dog_fs_test, y_dog_fs_train, y_dog_fs_test,
                 ada_dog,
                 "Optimised Adaboost", "Dog adoptions - selected features")

********************************
AdaBoostRegressor(learning_rate=0.1, loss='square', n_estimators=10,
                  random_state=32)
Optimised Adaboost  -  All adoptions - selected features
Shape of train data (42284, 15)
Shape of test data (18123, 15)
********************************
Model: Optimised Adaboost
Data: All adoptions - selected features
MAE Train: 0.589
MAE Test: 0.592
MSE Train: 0.537
MSE Test: 0.538
R-squared Train: 0.213
R-squared Test: 0.205
Time taken for modelling:  0.720149040222168

********************************
AdaBoostRegressor(learning_rate=0.1, loss='exponential', random_state=32)
Optimised Adaboost  -  Cat adoptions - selected features
Shape of train data (16907, 8)
Shape of test data (7247, 8)
********************************
Model: Optimised Adaboost
Data: Cat adoptions - selected features
MAE Train: 0.606
MAE Test: 0.603
MSE Train: 0.503
MSE Test: 0.502
R-squared Train: 0.164
R-squared Test: 0.165
Time taken for modelling:  0.9560165405273438

******

# Neural network

In [24]:
# define model
model =  MLPRegressor(random_state = seed)
# define search space
space = dict()
#space['learning_rate'] = [0.001, 0.003, 0.005]
space['learning_rate'] = ['constant', 'invscaling', 'adaptive']
space['activation'] = ['tanh','relu']
space['hidden_layer_sizes'] =[(20,20),(25,25), (30,30),(35,35), (40,40)]
#space['solver'] = ['lbfgs','sgd','adam'] #

print("*************All Adoptions***************")
print(X_fs_train.shape)
print(y_fs_train.shape)
mlp_all_best = GridCVforTuning(model, space, X_fs_train, y_fs_train)
                            
print("*************Cat Adoptions***************")
print(X_cat_fs_train.shape)
print(y_cat_fs_train.shape)
mlp_cat_best = GridCVforTuning(model, space, X_cat_fs_train, y_cat_fs_train)
                            
print("*************Dog Adoptions***************")
print(X_dog_fs_train.shape)
print(y_dog_fs_train.shape)
mlp_dog_best = GridCVforTuning(model, space, X_dog_fs_train, y_dog_fs_train)


*************All Adoptions***************
(42284, 15)
(42284,)
Identifying the best Hyperparameters.....
Best Score: 0.2382036068586212
Best Hyperparameters: {'activation': 'tanh', 'hidden_layer_sizes': (30, 30), 'learning_rate': 'constant'}
Time taken for modelling:  1015.8866231441498

*************Cat Adoptions***************
(16907, 8)
(16907,)
Identifying the best Hyperparameters.....
Best Score: 0.18539124180026167
Best Hyperparameters: {'activation': 'tanh', 'hidden_layer_sizes': (20, 20), 'learning_rate': 'constant'}
Time taken for modelling:  436.9930260181427

*************Dog Adoptions***************
(25377, 10)
(25377,)
Identifying the best Hyperparameters.....
Best Score: 0.17636953015105358
Best Hyperparameters: {'activation': 'tanh', 'hidden_layer_sizes': (35, 35), 'learning_rate': 'constant'}
Time taken for modelling:  5861.189352989197



In [25]:
mlp_all = MLPRegressor(learning_rate = 'constant', hidden_layer_sizes = (30,30), activation ='tanh')
mlp_cat = MLPRegressor(learning_rate = 'constant', hidden_layer_sizes = (20,20), activation ='tanh')
mlp_dog = MLPRegressor(learning_rate = 'constant', hidden_layer_sizes = (30,35), activation ='tanh')

In [26]:
#####################################################
#########Modelling on all features##############
#####################################################
PerformModelling(X_train, X_test, y_train, y_test , mlp_all,
                 "Optimised MLP NN", "All adoptions - selected features")
PerformModelling(X_cat_train, X_cat_test, y_cat_train, y_cat_test, mlp_cat,
                 "Optimised MLP NN", "Cat adoptions - selected features")
PerformModelling(X_dog_train, X_dog_test, y_dog_train, y_dog_test,
                 mlp_dog,
                 "Optimised MLP NN", "Dog adoptions - selected features")

********************************
MLPRegressor(activation='tanh', hidden_layer_sizes=(30, 30))
Optimised MLP NN  -  All adoptions - selected features
Shape of train data (42284, 22)
Shape of test data (18123, 22)
********************************
Model: Optimised MLP NN
Data: All adoptions - selected features
MAE Train: 0.578
MAE Test: 0.584
MSE Train: 0.515
MSE Test: 0.522
R-squared Train: 0.244
R-squared Test: 0.229
Time taken for modelling:  7.229928970336914

********************************
MLPRegressor(activation='tanh', hidden_layer_sizes=(20, 20))
Optimised MLP NN  -  Cat adoptions - selected features
Shape of train data (16907, 20)
Shape of test data (7247, 20)
********************************
Model: Optimised MLP NN
Data: Cat adoptions - selected features
MAE Train: 0.584
MAE Test: 0.581
MSE Train: 0.482
MSE Test: 0.483
R-squared Train: 0.199
R-squared Test: 0.197
Time taken for modelling:  2.492594003677368

********************************
MLPRegressor(activation='tanh', hidd

In [27]:
#####################################################
#########Modelling on selected features##############
#####################################################
PerformModelling(X_fs_train, X_fs_test, y_fs_train, y_fs_test , mlp_all,
                 "Optimised MLP NN", "All adoptions - selected features")
PerformModelling(X_cat_fs_train, X_cat_fs_test, y_cat_fs_train, y_cat_fs_test, mlp_cat,
                 "Optimised MLP NN", "Cat adoptions - selected features")
PerformModelling(X_dog_fs_train, X_dog_fs_test, y_dog_fs_train, y_dog_fs_test,
                 mlp_dog,
                 "Optimised MLP NN", "Dog adoptions - selected features")

********************************
MLPRegressor(activation='tanh', hidden_layer_sizes=(30, 30))
Optimised MLP NN  -  All adoptions - selected features
Shape of train data (42284, 15)
Shape of test data (18123, 15)
********************************
Model: Optimised MLP NN
Data: All adoptions - selected features
MAE Train: 0.567
MAE Test: 0.574
MSE Train: 0.496
MSE Test: 0.507
R-squared Train: 0.272
R-squared Test: 0.252
Time taken for modelling:  24.08254599571228

********************************
MLPRegressor(activation='tanh', hidden_layer_sizes=(20, 20))
Optimised MLP NN  -  Cat adoptions - selected features
Shape of train data (16907, 8)
Shape of test data (7247, 8)
********************************
Model: Optimised MLP NN
Data: Cat adoptions - selected features
MAE Train: 0.589
MAE Test: 0.585
MSE Train: 0.485
MSE Test: 0.484
R-squared Train: 0.193
R-squared Test: 0.195
Time taken for modelling:  2.6533989906311035

********************************
MLPRegressor(activation='tanh', hidde

# Random Forest

In [28]:
# define model
model =  RandomForestRegressor(random_state = seed)
# define search space
space = dict()
space['n_estimators'] = [100, 125, 150]
space['max_depth'] = [8, 10, 12]


print("*************All Adoptions***************")
print(X_fs_train.shape)
print(y_fs_train.shape)
rf_all_best = GridCVforTuning(model, space, X_fs_train, y_fs_train)
                            
print("*************Cat Adoptions***************")
print(X_cat_fs_train.shape)
print(y_cat_fs_train.shape)
rf_cat_best = GridCVforTuning(model, space, X_cat_fs_train, y_cat_fs_train)
                            
print("*************Dog Adoptions***************")
print(X_dog_fs_train.shape)
print(y_dog_fs_train.shape)
rf_dog_best = GridCVforTuning(model, space, X_dog_fs_train, y_dog_fs_train)



*************All Adoptions***************
(42284, 15)
(42284,)
Identifying the best Hyperparameters.....
Best Score: 0.4396017455859275
Best Hyperparameters: {'max_depth': 12, 'n_estimators': 150}
Time taken for modelling:  161.83353686332703

*************Cat Adoptions***************
(16907, 8)
(16907,)
Identifying the best Hyperparameters.....
Best Score: 0.3353281592378667
Best Hyperparameters: {'max_depth': 12, 'n_estimators': 125}
Time taken for modelling:  35.99373197555542

*************Dog Adoptions***************
(25377, 10)
(25377,)
Identifying the best Hyperparameters.....
Best Score: 0.44208112844550757
Best Hyperparameters: {'max_depth': 12, 'n_estimators': 150}
Time taken for modelling:  78.03726291656494



In [29]:
rf_all = RandomForestRegressor(max_depth = 12, n_estimators = 150)
rf_cat = RandomForestRegressor(max_depth = 12, n_estimators = 125)
rf_dog = RandomForestRegressor(max_depth = 12, n_estimators = 150)

In [30]:
#####################################################
#########Modelling on all features##############
#####################################################
PerformModelling(X_train, X_test, y_train, y_test , rf_all,
                 "Optimised MLP NN", "All adoptions - selected features")
PerformModelling(X_cat_train, X_cat_test, y_cat_train, y_cat_test, rf_cat,
                 "Optimised MLP NN", "Cat adoptions - selected features")
PerformModelling(X_dog_train, X_dog_test, y_dog_train, y_dog_test,
                 rf_dog,
                 "Optimised MLP NN", "Dog adoptions - selected features")

********************************
RandomForestRegressor(max_depth=12, n_estimators=150)
Optimised MLP NN  -  All adoptions - selected features
Shape of train data (42284, 22)
Shape of test data (18123, 22)
********************************
Model: Optimised MLP NN
Data: All adoptions - selected features
MAE Train: 0.402
MAE Test: 0.442
MSE Train: 0.312
MSE Test: 0.381
R-squared Train: 0.542
R-squared Test: 0.437
Time taken for modelling:  5.8488240242004395

********************************
RandomForestRegressor(max_depth=12, n_estimators=125)
Optimised MLP NN  -  Cat adoptions - selected features
Shape of train data (16907, 20)
Shape of test data (7247, 20)
********************************
Model: Optimised MLP NN
Data: Cat adoptions - selected features
MAE Train: 0.453
MAE Test: 0.504
MSE Train: 0.322
MSE Test: 0.409
R-squared Train: 0.464
R-squared Test: 0.319
Time taken for modelling:  1.5795631408691406

********************************
RandomForestRegressor(max_depth=12, n_estimators

In [31]:
#####################################################
#########Modelling on selected features##############
#####################################################
PerformModelling(X_fs_train, X_fs_test, y_fs_train, y_fs_test , rf_all,
                 "Optimised MLP NN", "All adoptions - selected features")
PerformModelling(X_cat_fs_train, X_cat_fs_test, y_cat_fs_train, y_cat_fs_test, rf_cat,
                 "Optimised MLP NN", "Cat adoptions - selected features")
PerformModelling(X_dog_fs_train, X_dog_fs_test, y_dog_fs_train, y_dog_fs_test,
                 rf_dog,
                 "Optimised MLP NN", "Dog adoptions - selected features")

********************************
RandomForestRegressor(max_depth=12, n_estimators=150)
Optimised MLP NN  -  All adoptions - selected features
Shape of train data (42284, 15)
Shape of test data (18123, 15)
********************************
Model: Optimised MLP NN
Data: All adoptions - selected features
MAE Train: 0.402
MAE Test: 0.442
MSE Train: 0.313
MSE Test: 0.382
R-squared Train: 0.541
R-squared Test: 0.436
Time taken for modelling:  5.121385812759399

********************************
RandomForestRegressor(max_depth=12, n_estimators=125)
Optimised MLP NN  -  Cat adoptions - selected features
Shape of train data (16907, 8)
Shape of test data (7247, 8)
********************************
Model: Optimised MLP NN
Data: Cat adoptions - selected features
MAE Train: 0.458
MAE Test: 0.501
MSE Train: 0.330
MSE Test: 0.406
R-squared Train: 0.452
R-squared Test: 0.324
Time taken for modelling:  1.1037299633026123

********************************
RandomForestRegressor(max_depth=12, n_estimators=15

/Users/surekha/opt/anaconda3/envs/DAAN570/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.84498e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/Users/surekha/opt/anaconda3/envs/DAAN570/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=9.29534e-19): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/Users/surekha/opt/anaconda3/envs/DAAN570/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=8.58858e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/Users/surekha/opt/anaconda3/envs/DAAN570/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.44712e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=Tr